In [1]:
%%bash

pip install kaggle segmentation_models_pytorch

#locate kaggle.json to /opt/ml/.kaggle/kaggle.json if you are using linux
mkdir data
cd data

kaggle datasets download -d perke986/face-mask-segmentation-dataset
unzip face-mask-segmentation-dataset.zip

rm face-mask-segmentation-dataset.zip


Archive:  face-mask-segmentation-dataset.zip
  inflating: Info.txt                
  inflating: images/coronavirus-4947340_1920.jpg  
  inflating: images/coronavirus-5064371_1920.jpg  
  inflating: images/mask-4898571_1920.jpg  
  inflating: images/mask-5136259_1920.jpg  
  inflating: images/mouth-guard-5060809_1920.jpg  
  inflating: images/mouth-guard-5068146_1920.jpg  
  inflating: images/nurse-4962034_1920.jpg  
  inflating: images/pexels-andrea-piacquadio-3881247.jpg  
  inflating: images/pexels-anna-shvets-3902881.jpg  
  inflating: images/pexels-anna-shvets-3943881.jpg  
  inflating: images/pexels-anna-shvets-3943882.jpg  
  inflating: images/pexels-anna-shvets-3943883.jpg  
  inflating: images/pexels-anna-shvets-3943884.jpg  
  inflating: images/pexels-anna-shvets-3943910.jpg  
  inflating: images/pexels-anna-shvets-3943952.jpg  
  inflating: images/pexels-anna-shvets-3943953.jpg  
  inflating: images/pexels-anna-shvets-3943954.jpg  
  inflating: images/pexels-anna-shvets-3962

100%|██████████| 78.9M/78.9M [00:04<00:00, 18.7MB/s]


In [2]:
import os
from collections import defaultdict

import cv2
from tqdm import tqdm

from dataset import get_DataLoader
from utils import *


seed = 42
seed_everything(seed)

In [3]:
all_path_dict = defaultdict(list)
get_all_path('data')

jpg_paths = all_path_dict['jpg']
png_paths = all_path_dict['png']
assert sorted(list(map(lambda x:x.split('.')[0].split('/')[-1], jpg_paths))) == sorted(list(map(lambda x:x.split('.')[0].split('/')[-1], png_paths))), 'pair does not match'

In [4]:
jpg_folder = 'data/images'
png_folder = 'data/masks'

for idx, jpg_file in enumerate(tqdm(os.listdir(jpg_folder))):
    jpg_path = os.path.join(jpg_folder, jpg_file)
    png_path = os.path.join(png_folder, jpg_file[:-3]+'png')
    
    rename_jpg_path = os.path.join(jpg_path.split('/')[0], str(idx) + '.jpg')
    rename_png_path = os.path.join(png_path.split('/')[0], str(idx) + '.png')
    
    os.rename(jpg_path, rename_jpg_path)
    os.rename(png_path, rename_png_path)
    
    mask = cv2.imread(rename_png_path).astype(bool).astype(np.uint8)
    
    mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
    cv2.imwrite(rename_png_path, mask)

100%|██████████| 222/222 [00:12<00:00, 18.17it/s]


In [5]:
%%bash

cd data
rm info.txt
rm -rf images
rm -rf masks
cd ..

rm: cannot remove 'info.txt': No such file or directory


## Now Everything is ready! Start Train!
`$python train.py`